In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/MNIST_data",one_hot=True)

Extracting /MNIST_data\train-images-idx3-ubyte.gz
Extracting /MNIST_data\train-labels-idx1-ubyte.gz
Extracting /MNIST_data\t10k-images-idx3-ubyte.gz
Extracting /MNIST_data\t10k-labels-idx1-ubyte.gz


In [2]:
# Parameters
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 28 # MNIST data input (img shape: 28*28)
n_steps = 28 # timesteps
n_hidden = 128 # hidden layer num of features
n_classes = 10 # MNIST total classes (0-9 digits)

In [3]:
X = tf.placeholder("float",[None,n_input,n_steps])
Y = tf.placeholder("float",[None,n_classes])


W = {"out":tf.Variable(tf.random_normal([n_hidden,n_classes]))}
b = {"out":tf.Variable(tf.random_normal([n_classes]))}


In [4]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    
    # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, n_steps, 1)
    
    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    
     # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [5]:
pred = RNN(X, W, b)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [7]:
# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

In [8]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
       # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={X: batch_x, Y: batch_y})
        
        
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={X: batch_x, Y: batch_y})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={X: batch_x, Y: batch_y})
            print ("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print ("Optimization Finished!")
        
    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print ("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))

Iter 1280, Minibatch Loss= 1.913677, Training Accuracy= 0.38281
Iter 2560, Minibatch Loss= 1.373407, Training Accuracy= 0.53906
Iter 3840, Minibatch Loss= 1.454489, Training Accuracy= 0.51562
Iter 5120, Minibatch Loss= 0.991461, Training Accuracy= 0.60938
Iter 6400, Minibatch Loss= 0.727245, Training Accuracy= 0.70312
Iter 7680, Minibatch Loss= 0.598043, Training Accuracy= 0.79688
Iter 8960, Minibatch Loss= 0.644258, Training Accuracy= 0.77344
Iter 10240, Minibatch Loss= 0.631338, Training Accuracy= 0.72656
Iter 11520, Minibatch Loss= 0.391768, Training Accuracy= 0.85156
Iter 12800, Minibatch Loss= 0.423948, Training Accuracy= 0.84375
Iter 14080, Minibatch Loss= 0.388162, Training Accuracy= 0.85156
Iter 15360, Minibatch Loss= 0.349105, Training Accuracy= 0.90625
Iter 16640, Minibatch Loss= 0.251928, Training Accuracy= 0.92188
Iter 17920, Minibatch Loss= 0.365567, Training Accuracy= 0.89844
Iter 19200, Minibatch Loss= 0.345845, Training Accuracy= 0.88281
Iter 20480, Minibatch Loss= 0.48